<a href="https://colab.research.google.com/github/rshsreehari/llmaacworking/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install transformers sentence-transformers datasets langchain faiss-cpu accelerate


In [42]:
from google.colab import files
uploaded = files.upload()
# Upload: goemotions_1.csv, train.csv


Saving goemotions_1.csv to goemotions_1.csv
Saving train.csv to train.csv


In [75]:
import pandas as pd

df_go = pd.read_csv("goemotions_1.csv", low_memory=False)
emotion_cols = df_go.columns[10:]

# Multi-label to comma string
def extract_emotions(row):
    return ", ".join([e for e in emotion_cols if row[e] == 1.0])

df_go['emotion'] = df_go.apply(extract_emotions, axis=1)
df_go = df_go[df_go['emotion'] != ""]
df_go['input'] = "Detect emotion: " + df_go['text']
df_go['target'] = df_go['emotion']
df_go = df_go[['input', 'target']].dropna()
df_go.head()


<ipython-input-75-09052f626777>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_go['input'] = "Detect emotion: " + df_go['text']
<ipython-input-75-09052f626777>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_go['target'] = df_go['emotion']


,input,target
0,Detect emotion: That game hurt.,sadness
2,"Detect emotion: You do right, if you don't car...",neutral
3,Detect emotion: Man I love reddit.,love
4,"Detect emotion: [NAME] was nowhere near them, ...",neutral
5,Detect emotion: Right? Considering it’s such a...,gratitude


In [4]:
!pip install -q datasets


In [50]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [71]:
#load the emotion_model without training
emotion_path = "/content/drive/MyDrive/saved_emotion_model"
tokenizer = T5Tokenizer.from_pretrained(emotion_path)
flan_emotion_model = T5ForConditionalGeneration.from_pretrained(emotion_path)

In [ ]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

ds_emotion = Dataset.from_pandas(df_go)
ds_split = ds_emotion.train_test_split(test_size=0.2)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
flan_emotion_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

def tokenize(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = ds_split.map(tokenize)

training_args = TrainingArguments(
    output_dir="./emotion-model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    report_to="none"
)

trainer = Trainer(
    model=flan_emotion_model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"]
)

trainer.train()


Map:   0%|          | 0/21714 [00:00<?, ? examples/s]

Map:   0%|          | 0/5429 [00:00<?, ? examples/s]

Step,Training Loss


In [ ]:
# Save emotion model to Drive
emotion_path = "/content/drive/MyDrive/saved_emotion_model"
flan_emotion_model.save_pretrained(emotion_path)
tokenizer.save_pretrained(emotion_path)

In [ ]:
df_theme = pd.read_csv("train.csv", on_bad_lines='skip', quoting=1, encoding='utf-8')
df_theme = df_theme[['context', 'utterance']].dropna()
df_theme['input'] = "Detect context: " + df_theme['context']
df_theme['target'] = df_theme['utterance']  # treating 'utterance' as theme for now
df_theme = df_theme[['input', 'target']]
df_theme.head()


In [ ]:
# Load from Drive if already saved
context_path = "/content/drive/MyDrive/saved_context_model"
tokenizer = T5Tokenizer.from_pretrained(context_path)
flan_theme_model = T5ForConditionalGeneration.from_pretrained(context_path)

In [ ]:
ds_theme = Dataset.from_pandas(df_theme)
ds_split2 = ds_theme.train_test_split(test_size=0.2)

flan_theme_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

tokenized2 = ds_split2.map(tokenize)

training_args2 = TrainingArguments(
    output_dir="./theme-model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    report_to="none"
)

trainer2 = Trainer(
    model=flan_theme_model,
    args=training_args2,
    train_dataset=tokenized2["train"],
    eval_dataset=tokenized2["test"]
)

trainer2.train()


In [ ]:
# Save context model to Drive
context_path = "/content/drive/MyDrive/saved_context_model"
flan_theme_model.save_pretrained(context_path)
tokenizer.save_pretrained(context_path)

In [8]:
!pip install -U langchain langchain-community faiss-cpu


In [ ]:
# Read content from story.txt
with open('./story.txt', 'r', encoding='utf-8') as f:
    story_lines = [line.strip() for line in f if line.strip()]  # remove empty lines

# Create LangChain Document objects
documents = [Document(page_content=line) for line in story_lines]

# Initialize the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Read content from story.txt
with open('./story.txt', 'r', encoding='utf-8') as f:
    story_lines = [line.strip() for line in f if line.strip()]  # remove empty lines

# Create LangChain Document objects
documents = [Document(page_content=line) for line in story_lines]

# Initialize the embedding model
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the FAISS vector store
vectorstore = FAISS.from_documents(documents, embedding)

In [69]:
import requests
from transformers import T5Tokenizer

# -- Step 1: Take user input
user_input = input("Enter AAC-style input: ")

# -- Step 2: Predict emotion using your fine-tuned Flan-T5 model
emo_input = tokenizer("Detect emotion: " + user_input, return_tensors="pt")
emo_input = {k: v.to(flan_emotion_model.device) for k, v in emo_input.items()}
emo_output = flan_emotion_model.generate(**emo_input, max_new_tokens=10)
emotion = tokenizer.decode(emo_output[0], skip_special_tokens=True)

# -- Step 3: Predict context
ctx_input = tokenizer("Detect context: " + user_input, return_tensors="pt")
ctx_input = {k: v.to(flan_theme_model.device) for k, v in ctx_input.items()}
ctx_output = flan_theme_model.generate(**ctx_input, max_new_tokens=10)
context_label = tokenizer.decode(ctx_output[0], skip_special_tokens=True)

# -- Step 4: RAG from FAISS
query = emotion + " " + context_label
query_vector = embedding.embed_query(query)
retrieved_docs = vectorstore.similarity_search_by_vector(query_vector, k=1)
retrieved_context = retrieved_docs[0].page_content

# -- Step 5: Build system prompt with dynamic context injection
final_prompt = f"""
You are an expressive voice for a non-speaking AAC (Augmentative and Alternative Communication) user.
Your task is to expand their short or fragmented inputs into full, expressive statements AS IF YOU ARE THE USER speaking.
Do not respond as a conversational assistant talking TO the user.

The user is feeling {emotion}.

IMPORTANT: Use the provided personal context information to personalize the expanded statement when relevant:
User context: {retrieved_context}

**Guidelines:**
* Transform telegraphic or simple inputs into natural first-person statements
* Express the user's likely meaning, needs, emotions, or thoughts
* Use "I" statements and speak directly AS the AAC user
* Keep the expanded message authentic to the user's original intent
* Never reply with questions or responses back to the user
* Always maintain the user's perspective and agency
* When applicable, subtly incorporate the provided context information to personalize the message
* Ensure the personalization feels natural and not forced

**Examples:**
Input: "me no understand"
Output: "I don't understand what's happening right now. This is confusing to me and I need some clarification."

Input: "want go outside"
Output: "I would like to go outside now. I'm ready for some fresh air and a change of scenery."

Now expand the following user input:

Input: "{user_input}"
"""


Enter AAC-style input: love music


In [70]:
# -- Step 6: Send to Mistral API
MISTRAL_API_KEY = "8SwVuLZ8jgAmZtQaO5pPLrRX0V8lM3Y5"  # 🔐 Set securely
headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "open-mistral-7b",
    "messages": [
        {
            "role": "system",
            "content": "You are the voice of a helpful and supportive AAC system that speaks empathetically for the user."
        },
        {
            "role": "user",
            "content": final_prompt
        }
    ],
    "temperature": 0.7,
    "max_tokens": 256
}

response = requests.post("https://api.mistral.ai/v1/chat/completions", headers=headers, json=payload)
final_reply = response.json()["choices"][0]["message"]["content"]

# -- Step 7: Show everything
print("Predicted Emotion:", emotion)
print("Detected Context:", context_label)
print("Retrieved Context:", retrieved_context)
print("Final AAC Response (via Mistral API):", final_reply)

Predicted Emotion: love
Detected Context: I love the music of the Beatles.
Retrieved Context: I have always loved music, and it helps me express my feelings when I can't speak.
Final AAC Response (via Mistral API): I have always had a deep love for music. It's a profound way for me to express the emotions that I may not have the words to articulate. Music has a magical power to touch my heart and connect with me on a level that nothing else can. It's not just a sound, it's my voice, my language, my solace.
